In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

In [42]:
titanik = sns.load_dataset('titanic')
titanik

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [43]:
titanik['embarked'].value_counts(),titanik['embark_town'].value_counts()

(embarked
 S    644
 C    168
 Q     77
 Name: count, dtype: int64,
 embark_town
 Southampton    644
 Cherbourg      168
 Queenstown      77
 Name: count, dtype: int64)

In [44]:
titanik['survived'].value_counts(),titanik['alive'].value_counts()

(survived
 0    549
 1    342
 Name: count, dtype: int64,
 alive
 no     549
 yes    342
 Name: count, dtype: int64)

In [45]:
titanik['sex'].value_counts(),titanik['who'].value_counts()

(sex
 male      577
 female    314
 Name: count, dtype: int64,
 who
 man      537
 woman    271
 child     83
 Name: count, dtype: int64)

In [46]:
titanik['pclass'].value_counts(),titanik['class'].value_counts()

(pclass
 3    491
 1    216
 2    184
 Name: count, dtype: int64,
 class
 Third     491
 First     216
 Second    184
 Name: count, dtype: int64)

In [47]:
titanik.drop(['embarked','alive','sex','pclass'], axis = 1, inplace=True)

In [56]:
titanik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   age          714 non-null    float64 
 2   sibsp        891 non-null    int64   
 3   parch        891 non-null    int64   
 4   fare         891 non-null    float64 
 5   class        891 non-null    category
 6   who          891 non-null    object  
 7   adult_male   891 non-null    bool    
 8   deck         203 non-null    category
 9   embark_town  889 non-null    object  
 10  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(2)
memory usage: 52.8+ KB


In [57]:
titanik.describe()

,survived,age,sibsp,parch,fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,29.699118,0.523008,0.381594,32.204208
std,0.486592,14.526497,1.102743,0.806057,49.693429
min,0.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,80.000000,8.000000,6.000000,512.329200


In [60]:
titanik.head(10)

,survived,age,sibsp,parch,fare,class,who,adult_male,deck,embark_town,alone
0,0,22.0,1,0,7.2500,Third,man,True,NaN,Southampton,False
1,1,38.0,1,0,71.2833,First,woman,False,C,Cherbourg,False
2,1,26.0,0,0,7.9250,Third,woman,False,NaN,Southampton,True
3,1,35.0,1,0,53.1000,First,woman,False,C,Southampton,False
4,0,35.0,0,0,8.0500,Third,man,True,NaN,Southampton,True
5,0,NaN,0,0,8.4583,Third,man,True,NaN,Queenstown,True
6,0,54.0,0,0,51.8625,First,man,True,E,Southampton,True
7,0,2.0,3,1,21.0750,Third,child,False,NaN,Southampton,False
8,1,27.0,0,2,11.1333,Third,woman,False,NaN,Southampton,False
9,1,14.0,1,0,30.0708,Second,child,False,NaN,Cherbourg,False


In [59]:
titanik.columns

Index(['survived', 'age', 'sibsp', 'parch', 'fare', 'class', 'who',
       'adult_male', 'deck', 'embark_town', 'alone'],
      dtype='object')

In [75]:
titanik['class'].value_counts()

class
Third     491
First     216
Second    184
Name: count, dtype: int64

In [51]:
titanik['deck'].value_counts()

deck
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: count, dtype: int64

In [71]:
titanik.groupby('deck', observed=True)['age'].mean()

deck
A    44.833333
B    34.955556
C    36.086667
D    39.032258
E    38.116667
F    19.954545
G    14.750000
Name: age, dtype: float64

In [77]:
pd.crosstab(titanik['deck'], titanik['class'])

class,First,Second,Third
deck,,,
A,15,0,0
B,47,0,0
C,59,0,0
D,29,4,0
E,25,4,3
F,0,8,5
G,0,0,4


In [80]:
pd.crosstab(titanik['deck'], titanik['who'])

who,child,man,woman
deck,,,
A,1,13,1
B,3,19,25
C,2,31,26
D,0,15,18
E,1,16,15
F,4,5,4
G,2,0,2


In [85]:
x = titanik[['deck','who','class','fare','age']]


,deck,who,class,fare,age
0,NaN,man,Third,7.2500,22.0
1,C,woman,First,71.2833,38.0
2,NaN,woman,Third,7.9250,26.0
3,C,woman,First,53.1000,35.0
4,NaN,man,Third,8.0500,35.0
...,...,...,...,...,...
886,NaN,man,Second,13.0000,27.0
887,B,woman,First,30.0000,19.0
888,NaN,woman,Third,23.4500,NaN
889,C,man,First,30.0000,26.0


In [104]:
OrdinalEncoder(categories=

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [103]:
x

,deck,who,class,fare,age
0,NaN,man,Third,7.2500,22.0
1,C,woman,First,71.2833,38.0
2,NaN,woman,Third,7.9250,26.0
3,C,woman,First,53.1000,35.0
4,NaN,man,Third,8.0500,35.0
...,...,...,...,...,...
886,NaN,man,Second,13.0000,27.0
887,B,woman,First,30.0000,19.0
888,NaN,woman,Third,23.4500,NaN
889,C,man,First,30.0000,26.0


In [94]:
x['class']

0       Third
1       First
2       Third
3       First
4       Third
        ...  
886    Second
887     First
888     Third
889     First
890     Third
Name: class, Length: 891, dtype: category
Categories (3, object): ['First', 'Second', 'Third']

In [ ]:
Pipeline('ordinal', Ordinal

In [88]:
imputer = KNNImputer(n_neighbors= 5, weights='uniform')
imputer.fit_transform(x)

ValueError: could not convert string to float: 'C'

In [82]:
KNNImputer?

Init signature:
KNNImputer(
    *,
    missing_values=nan,
    n_neighbors=5,
    weights='uniform',
    metric='nan_euclidean',
    copy=True,
    add_indicator=False,
    keep_empty_features=False,
)
Docstring:     
Imputation for completing missing values using k-Nearest Neighbors.

Each sample's missing values are imputed using the mean value from
`n_neighbors` nearest neighbors found in the training set. Two samples are
close if the features that neither is missing are close.

Read more in the :ref:`User Guide <knnimpute>`.

.. versionadded:: 0.22

Parameters
----------
missing_values : int, float, str, np.nan or None, default=np.nan
    The placeholder for the missing values. All occurrences of
    `missing_values` will be imputed. For pandas' dataframes with
    nullable integer dtypes with missing values, `missing_values`
    should be set to np.nan, since `pd.NA` will be converted to np.nan.

n_neighbors : int, default=5
    Number of neighboring samples to use for imputation.